In [1]:
from edbo.plus.optimizer_botorch import EDBOplus
from edbo.edbo.feature_utils import reaction_space

C:\Anaconda3\envs\edbo\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


#### This is a tutorial that covers the basics for running EDBO+: from designing a combinatorial space to running the Bayesian Optimizer.

## 1. Creating a search scope using EDBO+.

##### To run EDBO+ we need to first create a reaction scope (search space) in a .csv format with all the possible combinations that you want to consider for our optimization. 
##### You can "manually" create a reaction scope using any spreadsheet editor (such as Excel, Libreoffice Calc, ...) but we have also created a tool to help you generating combinatorial search spaces. 
##### For instance, lets say that we want to consider 3 solvents ($\bf{THF}$, $\bf{Toluene}$, $\bf{DMSO}$), 4 temperatures ($\bf{-10}$, $\bf{0}$, $\bf{10}$, $\bf{25}$) and 3 concentration levels ($\bf{0.1}$, $\bf{0.2}$, $\bf{1.0}$). We can introduce these in the EDBO+ scope generator in a dictionary form as follows:

In [2]:
import pandas as pd
from edbo.edbo.utils import Data

# Load DFT descriptor CSV files computed with auto-qchem using pandas
# Instantiate a Data object

azadicarbs = Data(pd.read_csv('../mitsunobu_optimization/descriptors/azadicarboxylate_boltzmann_dft.csv'))
phosphines = Data(pd.read_csv('../mitsunobu_optimization/descriptors/phosphine_boltzmann_dft.csv'))
solvents = Data(pd.read_csv('../mitsunobu_optimization/descriptors/solvent_dft.csv'))

# Use Data.drop method to drop descriptors containing some unwanted keywords

for data in [azadicarbs, phosphines, solvents]:
    data.drop(['file_name', 'entry', 'vibration', 'correlation', 'Rydberg', 
               'correction', 'atom_number', 'E-M_angle', 'MEAN', 'MAXG', 
               'STDEV'])

In [3]:
azadicarbs.data

azadicarboxylate_SMILES  \
0                  CC(C)(OC(/N=N/C(OC(C)(C)C)=O)=O)C   
1                        CC(OC(/N=N/C(OC(C)C)=O)=O)C   
2  ClC1=CC=C(COC(/N=N/C(OCC2=CC=C(Cl)C=C2)=O)=O)C=C1   
3          O=C(OCC1=CC=CC=C1)/N=N/C(OCC2=CC=CC=C2)=O   
4                    O=C(N1CCCCC1)/N=N/C(N2CCCCC2)=O   
5                          CN(C(/N=N/C(N(C)C)=O)=O)C   

   azadicarboxylate_number_of_atoms_MING  \
0                                     34   
1                                     28   
2                                     36   
3                                     36   
4                                     38   
5                                     24   

   azadicarboxylate_number_of_atoms_Boltz  azadicarboxylate_charge_MING  \
0                                      34                             0   
1                                      28                             0   
2                                      36                             0   
3                                      36                             0   
4                                      38                             0   
5                                      24                             0   

   azadicarboxylate_charge_Boltz  azadicarboxylate_multiplicity_MING  \
0                              0                                   1   
1                              0                                   1   
2                              0                                   1   
3                              0                                   1   
4                              0                                   1   
5                              0                                   1   

   azadicarboxylate_multiplicity_Boltz  azadicarboxylate_dipole_MING  \
0                                    1                        0.0004   
1                                    1                        3.8641   
2                                    1                        4.1587   
3                                    1                        1.2714   
4                                    1                        0.0000   
5                                    1                        0.0009   

   azadicarboxylate_dipole_Boltz  azadicarboxylate_molar_mass_MING  ...  \
0                       1.939532                          230.2632  ...   
1                       2.192121                          202.2096  ...   
2                       2.959765                          367.1878  ...   
3                       2.214453                          298.2976  ...   
4                       0.274424                          252.3156  ...   
5                       0.000600                          172.1864  ...   

   azadicarboxylate_atom8_%VBur_MING  azadicarboxylate_atom8_%VBur_Boltz  \
0                           0.655938                            0.655539   
1                           0.601719                            0.607769   
2                           0.566270                            0.567675   
3                           0.568606                            0.570347   
4                           0.719739                            0.720996   
5                           0.646388                            0.646453   

   azadicarboxylate_c_min_%VBur_MING  azadicarboxylate_c_min_%VBur_Boltz  \
0                           0.655909                            0.655540   
1                           0.601719                            0.607747   
2                           0.558696                            0.565456   
3                           0.577079                            0.571747   
4                           0.420047                            0.420409   
5                           0.418760                            0.418853   

   azadicarboxylate_c_min+1_%VBur_MING  azadicarboxylate_c_min+1_%VBur_Boltz  \
0                             0.655938                              0.655542   
1                             0.598994 

In [4]:
phosphines.data

phosphine_SMILES  \
0              C1(P(C2=CC=CC=C2)C3=CC=CC=C3)=CC=CC=C1   
1                                     CCCCP(CCCC)CCCC   
2                          CP(C1=CC=CC=C1)C2=CC=CC=C2   
3    COC1=CC=C(P(C2=CC=C(OC)C=C2)C3=CC=C(OC)C=C3)C=C1   
4       FC1=CC=C(P(C2=CC=C(F)C=C2)C3=CC=C(F)C=C3)C=C1   
5   FC(F)(F)C1=CC=C(P(C2=CC=C(C(F)(F)F)C=C2)C3=CC=...   
6       CC1=CC=C(P(C2=CC=C(C=C2)C)C3=CC=C(C=C3)C)C=C1   
7                       C1(P(C2CCCCC2)C3CCCCC3)CCCCC1   
8                    C1(P(C2=CC=CO2)C3=CC=CO3)=CC=CO1   
9             CC1=CC=C(P(C2=CC=CC=C2)C3=CC=CC=C3)C=C1   
10   ClC1=CC=C(P(C2=CC=C(Cl)C=C2)C3=CC=C(Cl)C=C3)C=C1   
11        CN(C1=CC=C(P(C2=CC=CC=C2)C3=CC=CC=C3)C=C1)C   

    phosphine_number_of_atoms_MING  phosphine_number_of_atoms_Boltz  \
0                               34                               34   
1                               40                               40   
2                               27                               27   
3                               46                               46   
4                               34                               34   
5                               43                               43   
6                               43                               43   
7                               52                               52   
8                               25                               25   
9                               37                               37   
10                              34                               34   
11                              42                               42   

    phosphine_charge_MING  phosphine_charge_Boltz  \
0                       0                       0   
1                       0                       0   
2                       0                       0   
3                       0                       0   
4                       0                       0   
5                       0                       0   
6                       0                       0   
7                       0                       0   
8                       0                       0   
9                       0                       0   
10                      0                       0   
11                      0                       0   

    phosphine_multiplicity_MING  phosphine_multiplicity_Boltz  \
0                             1                             1   
1                             1                             1   
2                             1                             1   
3                             1                             1   
4                             1                             1   
5                             1                             1   
6                             1                             1   
7                             1                             1   
8                             1                             1   
9                             1                             1   
10                            1                             1   
11                            1                             1   

    phosphine_dipole_MING  phosphine_dipole_Boltz  phosphine_molar_mass_MING  \
0                  1.4132                1.413200                   262.2903   
1                  1.2127                1.262996                   202.3191   
2                  1.3922                1.392118                   200.2195   
3                  1.9316                2.588852                   352.3689   
4                  0.0703                0.070498                   316.2618   
5                  1.4571                1.406445                   466.2852   
6                  1.6967                1.697445                   304.3707   
7                  1.1313                1.131300                   280.4325   
8                  1.5585                1.740011                   232.1751   
9                  1.6036          

In [5]:
solvents.data

solvent_SMILES solvent_stoichiometry  solvent_number_of_atoms  \
0       O=CN(C)C                C3H7NO                       12   
1        C1CCOC1                 C4H8O                       13   
2           CC#N                 C2H3N                        6   
3   CC1=CC=CC=C1                  C7H8                       15   
4      CC(OCC)=O                C4H8O2                       14   

   solvent_charge  solvent_multiplicity solvent_convergence_criteria  \
0               0                     1                          met   
1               0                     1                          met   
2               0                     1                          met   
3               0                     1                          met   
4               0                     1                          met   

   solvent_dipole  solvent_molar_mass  solvent_molar_volume  \
0          3.8297             73.0944               535.244   
1          1.7892             72.1066               707.723   
2          3.8134             41.0524               486.058   
3          0.3196             92.1402              1020.231   
4          1.7645             88.1060               794.570   

   solvent_electronic_spatial_extent  ...  solvent_c_max-1_NMR_anisotropy  \
0                           443.7717  ...                          6.5332   
1                           373.6312  ...                         69.8309   
2                           161.6325  ...                          8.5261   
3                           719.3373  ...                          7.1860   
4                           660.2399  ...                          6.0087   

   solvent_c_max-1_ES_root_Mulliken_charge  \
0                                 0.178210   
1                                -0.044254   
2                                 0.186182   
3                                 0.157341   
4                                 0.161465   

   solvent_c_max-1_ES_root_NPA_charge  solvent_c_max-1_ES_root_NPA_core  \
0                             0.24000                           0.00000   
1                            -0.16679                           1.99922   
2                             0.25269                           0.00000   
3                             0.23884                           0.00000   
4                             0.23117                           0.00000   

   solvent_c_max-1_ES_root_NPA_valence  solvent_c_max-1_ES_root_NPA_total  \
0                              0.75891                            0.76000   
1                              4.14509                            6.16679   
2                              0.74074                            0.74731   
3                              0.75956                            0.76116   
4                              0.76438                            0.76883   

   solvent_c_min_%VBur  solvent_c_min+1_%VBur  solvent_c_max_%VBur  \
0             0.355169               0.616419             0.467022   
1             0.499296               0.531780             0.500853   
2             0.349151               0.283974             0.398371   
3             0.425227               0.540253             0.351726   
4             0.557379               0.431125             0.558247   

   solvent_c_max-1_%VBur  
0               0.390228  
1               0.500704  
2               0.304392  
3               0.351816  
4               0.334481  

[5 rows x 122 columns]

In [9]:
## generating reaction space

# Parameters in reaction space

components = {'azadicarboxylate':'DFT',                             # DFT descriptors
              'phosphine':'DFT',                                    # DFT descriptors
              'solvent':'DFT',                                      # DFT descriptors
              'substrate_concentration':[0.05, 0.10, 0.15, 0.20],   # Discrete grid of concentrations
              'azadicarb_equiv':[1.1, 1.3, 1.5, 1.7, 1.9],          # Discrete grid of equiv.
              'phos_equiv':[1.1, 1.3, 1.5, 1.7, 1.9],               # Discrete grid of equiv.
              'temperature':[5, 15, 25, 35, 45]}                    # Discrete grid of temperatures

# Encodings - if not specified EDBO will automatically use OHE

encoding = {'substrate_concentration':'numeric',                    # Numerical encoding
            'azadicarb_equiv':'numeric',                            # Numerical encoding
            'phos_equiv':'numeric',                                 # Numerical encoding
            'temperature':'numeric'}                                # Numerical encoding

# External descriptor matrices override specified encoding

dft = {'azadicarboxylate':azadicarbs.data,                          # Unprocessed descriptor DataFrame
       'phosphine':phosphines.data,                                 # Unprocessed descriptor DataFrame
       'solvent':solvents.data}                                     # Unprocessed descriptor DataFrame

df_scope = reaction_space(components, 
                    encoding=encoding,
                    descriptor_matrices=dft)
df_scope.data.to_csv('my_optimization_test.csv', index=False)

In [2]:
#reaction_components = {
#    'solvent': ['THF', 'Toluene', 'DMSO'],
#    'T': [-10, 0, 10, 25],
#    'concentration': [0.1, 0.2, 1.0]
#}

##### Now we need to pass the previous dictionary to the $\bf{generate\_reaction\_scope}$ tool in the EDBOplus class.

In [3]:
#EDBOplus().generate_reaction_scope(
#    components=reaction_components, 
#    filename='my_optimization.csv',
#    check_overwrite=False
#)

Generating reaction scope...


solvent   T  concentration
0       THF -10            0.1
1       THF -10            0.2
2       THF -10            1.0
3       THF   0            0.1
4       THF   0            0.2
5       THF   0            1.0
6       THF  10            0.1
7       THF  10            0.2
8       THF  10            1.0
9       THF  25            0.1
10      THF  25            0.2
11      THF  25            1.0
12  Toluene -10            0.1
13  Toluene -10            0.2
14  Toluene -10            1.0
15  Toluene   0            0.1
16  Toluene   0            0.2
17  Toluene   0            1.0
18  Toluene  10            0.1
19  Toluene  10            0.2
20  Toluene  10            1.0
21  Toluene  25            0.1
22  Toluene  25            0.2
23  Toluene  25            1.0
24     DMSO -10            0.1
25     DMSO -10            0.2
26     DMSO -10            1.0
27     DMSO   0            0.1
28     DMSO   0            0.2
29     DMSO   0            1.0
30     DMSO  10            0.1
31     DMSO  10            0.2
32     DMSO  10            1.0
33     DMSO  25            0.1
34     DMSO  25            0.2
35     DMSO  25            1.0

##### We can always load/read the previously generated reaction scope using any spreadsheet editor but in this case we will use Pandas for that:

In [12]:
import pandas as pd
df_scope = pd.read_csv('my_optimization_test.csv')  # Load csv file.

##### Now we can check the number of combinations in the reaction scope:

In [13]:
n_combinations = len(df_scope)
print(f"Your reaction scope has {n_combinations} combinations.")

Your reaction scope has 180000 combinations.


##### Of course, this is a very small reaction scope  as it is meant to be a toy model to demonstrate how EDBO+ works.

## 2. First steps, initializing EDBO+ (in absence of training data).

##### We are going to execute EDBO+ to suggest some initial samples. 
##### Since we have not collected any experimental data (observations) yet, EDBO+ will suggest a set of initial experiments based on an feature space sampling method, in this case the CVT sampling method (see:http://kmh-lanl.hansonhub.com/uncertainty/meetings/gunz03vgr.pdf).
##### In this example the $\bf{objective}$ is to maximize the reaction $\bf{yield}$ and $\bf{enantioselectivity}$ but at the same time we want to minimize the amount one of a given $\bf{side~product}$ in this reaction. We also need to introduce the name of the csv file containing our reaction scope (in our case this was $\bf{my\_optimization.csv}$). Now we can execute the algorithm using the $\bf{run}$ command in EDBOplus:

In [14]:
EDBOplus().run(
    filename='my_optimization_test.csv',  # Previously generated scope.
    objectives=['yield', 'ee', 'side_product'],  # Objectives to be optimized.
    objective_mode=['max', 'max', 'min'],  # Maximize yield and ee but minimize side_product.
    batch=3,  # Number of experiments in parallel that we want to perform in this round.
    columns_features='all', # features to be included in the model.
    init_sampling_method='cvtsampling'  # initialization method.
)

Sampling type:  selection 


Number of unique samples returned by sampling algorithm: 3
Creating a priority list using random sampling: cvtsampling


azadicarboxylate_number_of_atoms_MING  azadicarboxylate_dipole_MING  \
105314                               0.857143                      0.305721   
105189                               0.857143                      0.305721   
112688                               0.857143                      0.305721   
120003                               1.000000                      0.000000   
119995                               0.857143                      0.305721   
...                                       ...                           ...   
60001                                0.857143                      1.000000   
60002                                0.857143                      1.000000   
60003                                0.857143                      1.000000   
60004                                0.857143                      1.000000   
179999                               0.000000                      0.000216   

        azadicarboxylate_dipole_Boltz  azadicarboxylate_molar_mass_MING  \
105314                       0.748134                          0.646719   
105189                       0.748134                          0.646719   
112688                       0.748134                          0.646719   
120003                       0.092534                          0.410916   
119995                       0.748134                          0.646719   
...                               ...                               ...   
60001                        1.000000                          1.000000   
60002                        1.000000                          1.000000   
60003                        1.000000                          1.000000   
60004                        1.000000                          1.000000   
179999                       0.000000                          0.000000   

        azadicarboxylate_molar_volume_MING  azadicarboxylate_lumo_energy_MING  \
105314                            0.691761                           0.422153   
105189                            0.691761                           0.422153   
112688                            0.691761                           0.422153   
120003                            0.721708                           1.000000   
119995                            0.691761                           0.422153   
...                                    ...                                ...   
60001                             1.000000                           0.000000   
60002                             1.000000                           0.000000   
60003                             1.000000                           0.000000   
60004                             1.000000                           0.000000   
179999                            0.000000                           0.782918   

        azadicarboxylate_hardness_MING  azadicarboxylate_hardness_Boltz  \
105314                        0.714081                         0.767235   
105189                        0.714081                         0.767235   
112688                        0.714081                         0.767235   
120003                        0.000000                         0.135560   
119995                        0.714081                         0.767235   
...                                ...                              ...   
60001                         0.369550                         0.447016   
60002                         0.369550                         0.447016   
60003                         0.369550                         0.447016   
60004                         0.369550                         0.447016   
179999                        0.002144                         0.000000   

        azadicarboxylate_E_scf_MING  \
105314                     0.684741   
105189                     0.684741   
112688                     0.684741   
120003                     0.826069   
119995                     0.684741   
...                             ...   
60001      

##### EDBO+ has created a column for each objective and added $\bf{PENDING}$ values to all of them so you can track the experiments that you have been collecting during the optimization campaign.
##### We can also see that EDBO+ has created a new $\bf{priority}$ column. This column is used to distinguish between high and low priority samples. The top entries (with $\bf{priority=1}$) highlight the next suggested samples.

##### We can check now the first 5 experiments in the scope by reading the $\bf{my\_optimization.csv}$ file:

In [15]:
df_edbo = pd.read_csv('my_optimization_test.csv')
df_edbo.head(5)

azadicarboxylate_number_of_atoms_MING  azadicarboxylate_dipole_MING  \
0                               0.857143                      0.305721   
1                               0.857143                      0.305721   
2                               0.857143                      0.305721   
3                               1.000000                      0.000000   
4                               0.857143                      0.305721   

   azadicarboxylate_dipole_Boltz  azadicarboxylate_molar_mass_MING  \
0                       0.748134                          0.646719   
1                       0.748134                          0.646719   
2                       0.748134                          0.646719   
3                       0.092534                          0.410916   
4                       0.748134                          0.646719   

   azadicarboxylate_molar_volume_MING  azadicarboxylate_lumo_energy_MING  \
0                            0.691761                           0.422153   
1                            0.691761                           0.422153   
2                            0.691761                           0.422153   
3                            0.721708                           1.000000   
4                            0.691761                           0.422153   

   azadicarboxylate_hardness_MING  azadicarboxylate_hardness_Boltz  \
0                        0.714081                         0.767235   
1                        0.714081                         0.767235   
2                        0.714081                         0.767235   
3                        0.000000                         0.135560   
4                        0.714081                         0.767235   

   azadicarboxylate_E_scf_MING  azadicarboxylate_ES_root_molar_volume_MING  \
0                     0.684741                                    0.923161   
1                     0.684741                                    0.923161   
2                     0.684741                                    0.923161   
3                     0.826069                                    1.000000   
4                     0.684741                                    0.923161   

   ...  solvent_c_min_%VBur  solvent_c_min+1_%VBur  substrate_concentration  \
0  ...               0.0289               1.000000                 0.666667   
1  ...               0.0289               1.000000                 0.333333   
2  ...               0.0289               1.000000                 0.333333   
3  ...               0.0289               1.000000                 0.000000   
4  ...               1.0000               0.442633                 1.000000   

   azadicarb_equiv  phos_equiv  temperature    yield       ee  side_product  \
0              0.5         0.5         1.00  PENDING  PENDING       PENDING   
1              0.5         0.5         1.00  PENDING  PENDING       PENDING   
2              0.5         0.5         0.75  PENDING  PENDING       PENDING   
3              0.0         0.0         0.75  PENDING  PENDING       PENDING   
4              1.0         1.0         0.00  PENDING  PENDING       PENDING   

   priority  
0         1  
1         1  
2         1  
3         0  
4         0  

[5 rows x 282 columns]

## 3. Adding training data in EDBO+.

##### Note: We will use Python and Pandas to add new training data in this example. But you can always edit and add new data into the '.csv' file using any spreedsheet editor (such as Excel, Libreoffice Calc, ...) if that's more convinient for you.

##### Let's open again the $\bf{my\_optimization.csv}$ file we generated before:

In [12]:
df_edbo = pd.read_csv('my_optimization.csv')
df_edbo.head(5)

solvent   T  concentration    yield       ee side_product  priority
0     DMSO  10            1.0  PENDING  PENDING      PENDING         1
1      THF  10            1.0  PENDING  PENDING      PENDING         1
2  Toluene  10            0.2  PENDING  PENDING      PENDING         1
3      THF -10            0.1  PENDING  PENDING      PENDING         0
4     DMSO -10            1.0  PENDING  PENDING      PENDING         0

##### We can fill the first out entry in the previous dataframe with the "observed" values using Pandas:

In [13]:
df_edbo.loc[0, 'yield'] = 20.5
df_edbo.loc[0, 'ee'] = 40
df_edbo.loc[0, 'side_product'] = 0.1

##### We can check that we have filled out the first entry with our "observed data":

In [14]:
df_edbo.head(5)

solvent   T  concentration    yield       ee side_product  priority
0     DMSO  10            1.0     20.5       40          0.1         1
1      THF  10            1.0  PENDING  PENDING      PENDING         1
2  Toluene  10            0.2  PENDING  PENDING      PENDING         1
3      THF -10            0.1  PENDING  PENDING      PENDING         0
4     DMSO -10            1.0  PENDING  PENDING      PENDING         0

##### We can also fill out the second entry with their corresponding "observations":

In [15]:
df_edbo.loc[1, 'yield'] = 50.3
df_edbo.loc[1, 'ee'] = 10
df_edbo.loc[1, 'side_product'] = 0.2

In [16]:
df_edbo.head(5)

solvent   T  concentration    yield       ee side_product  priority
0     DMSO  10            1.0     20.5       40          0.1         1
1      THF  10            1.0     50.3       10          0.2         1
2  Toluene  10            0.2  PENDING  PENDING      PENDING         1
3      THF -10            0.1  PENDING  PENDING      PENDING         0
4     DMSO -10            1.0  PENDING  PENDING      PENDING         0

##### Now we can save our dataset as $\bf{my\_optimization\_round0.csv}$:

In [17]:
df_edbo.to_csv('my_optimization_round0.csv', index=False)

## 4. Running EDBO+ with training data.

##### First let's check our previous data (which include some $\bf{yield}$, $\bf{ee}$ and $\bf{side\_product}$ observations, which will be used to train the model):

In [18]:
df_edbo_round0 = pd.read_csv('my_optimization_round0.csv')
df_edbo_round0.head(5)

solvent   T  concentration    yield       ee side_product  priority
0     DMSO  10            1.0     20.5       40          0.1         1
1      THF  10            1.0     50.3       10          0.2         1
2  Toluene  10            0.2  PENDING  PENDING      PENDING         1
3      THF -10            0.1  PENDING  PENDING      PENDING         0
4     DMSO -10            1.0  PENDING  PENDING      PENDING         0

##### Now that we have introduced some "observations" in our $\bf{my\_optimization\_round0.csv}$ file, we can execute EDBO+ to suggest samples using these "observations" as training data.

In [19]:
EDBOplus().run(
    filename='my_optimization_round0.csv',  # Previous scope (including observations).
    objectives=['yield', 'ee', 'side_product'],  # Objectives to be optimized.
    objective_mode=['max', 'max', 'min'],  # Maximize yield and ee but minimize side_product.
    batch=3,  # Number of experiments in parallel that we want to perform in this round.
    columns_features='all', # features to be included in the model.
    init_sampling_method='cvtsampling'  # initialization method.
)

The following columns are categorical and will be encoded using One-Hot-Encoding: ['solvent']
Using EHVI acquisition function.
Using hyperparameters optimized for continuous variables.
Using hyperparameters optimized for continuous variables.
Using hyperparameters optimized for continuous variables.
Number of QMC samples using SobolQMCNormalSampler sampler: 512
Acquisition function optimized.
Predictions obtained and expected improvement obtained.


solvent   T  concentration    yield       ee side_product  priority
19      THF -10            0.2  PENDING  PENDING      PENDING       1.0
35     DMSO  25            1.0  PENDING  PENDING      PENDING       1.0
10     DMSO   0            0.2  PENDING  PENDING      PENDING       1.0
7   Toluene  25            1.0  PENDING  PENDING      PENDING       0.0
6   Toluene  25            0.2  PENDING  PENDING      PENDING       0.0
5   Toluene  25            0.1  PENDING  PENDING      PENDING       0.0
17  Toluene  10            1.0  PENDING  PENDING      PENDING       0.0
2   Toluene  10            0.2  PENDING  PENDING      PENDING       0.0
18  Toluene  10            0.1  PENDING  PENDING      PENDING       0.0
28  Toluene   0            1.0  PENDING  PENDING      PENDING       0.0
34  Toluene   0            0.2  PENDING  PENDING      PENDING       0.0
33  Toluene   0            0.1  PENDING  PENDING      PENDING       0.0
32  Toluene -10            1.0  PENDING  PENDING      PENDING       0.0
31  Toluene -10            0.2  PENDING  PENDING      PENDING       0.0
30  Toluene -10            0.1  PENDING  PENDING      PENDING       0.0
29      THF  25            1.0  PENDING  PENDING      PENDING       0.0
27      THF  25            0.2  PENDING  PENDING      PENDING       0.0
20      THF  25            0.1  PENDING  PENDING      PENDING       0.0
26      THF  10            0.2  PENDING  PENDING      PENDING       0.0
25      THF  10            0.1  PENDING  PENDING      PENDING       0.0
24      THF   0            1.0  PENDING  PENDING      PENDING       0.0
23      THF   0            0.2  PENDING  PENDING      PENDING       0.0
22      THF   0            0.1  PENDING  PENDING      PENDING       0.0
21      THF -10            1.0  PENDING  PENDING      PENDING       0.0
3       THF -10            0.1  PENDING  PENDING      PENDING       0.0
16     DMSO  25            0.2  PENDING  PENDING      PENDING       0.0
15     DMSO  25            0.1  PENDING  PENDING      PENDING       0.0
14     DMSO  10            0.2  PENDING  PENDING      PENDING       0.0
13     DMSO  10            0.1  PENDING  PENDING      PENDING       0.0
12     DMSO   0            1.0  PENDING  PENDING      PENDING       0.0
11     DMSO   0            0.1  PENDING  PENDING      PENDING       0.0
4      DMSO -10            1.0  PENDING  PENDING      PENDING       0.0
9      DMSO -10            0.2  PENDING  PENDING      PENDING       0.0
8      DMSO -10            0.1  PENDING  PENDING      PENDING       0.0
1       THF  10            1.0     50.3       10          0.2      -1.0
0      DMSO  10            1.0     20.5       40          0.1      -1.0

##### Again the samples suggested by EDBO+ have $\bf{priority = +1}$. In addition, we asign $\bf{priority = -1}$ to the experiments that we have already run (these are at the bottom of the dataset).

## Extra: Accessing the model predictions.

##### Each time that EDBO+ is executed with training data it will generate a .csv file with the predictions for the entire scope (including the 'untested' samples).
##### In the previous example, by running EDBO+ with training data, we generated two files: $\bf{my\_optimization\_round0.csv}$ and a second file with the predictions $\bf{pred\_my\_optimization\_round0.csv}$.
##### Let's have a look to the predictions file using Pandas:

In [16]:
df_predictions_round0 = pd.read_csv('pred_my_optimization_round0.csv')
df_predictions_round0.style.background_gradient(subset=['priority'], cmap='plasma')

##### In the previous dataset we can access the model predictions ($\bf{predicted\_mean}$ and $\bf{predicted\_variance}$ columns) but also to the expected improvement values (in the $\bf{expected\_improvement}$ columns) for each objective ($\bf{yield}$, $\bf{ee}$ and $\bf{side\_product}$.).